In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/kw-ai'

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras import backend as K

In [ ]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join(path, 'data/train_features.csv')
y_train_path = os.path.join(path, 'data/train_labels.csv')
x_test_path = os.path.join(path, 'data/test_features.csv')
sub_path = os.path.join(path, 'data/sample_submission.csv')

x_train = pd.read_csv(x_train_path)
y_train = pd.read_csv(y_train_path)
x_test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [ ]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [ ]:
series_train = make_dataset(x_train)
series_test = make_dataset(x_test)

In [ ]:
series_train.shape, series_test.shape

((3125, 600, 6), (782, 600, 6))

In [ ]:
# label one-hot 인코딩
labels = y_train['label'].to_numpy()

In [ ]:
import tensorflow as tf

In [ ]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 128
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000, seed=42)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 128
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [ ]:
from tensorflow import keras

In [ ]:
# 가중치 적용 loss
# https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
# 위 논문 참고
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

# 모델을 만들어 주는 함수
# 기존 base에서 overfitting이 심해, dropout을 늘림(아직 제출은 안해봄)
def base():
  model = keras.models.Sequential([
            keras.layers.Conv1D(128, 64, input_shape=[600, 6]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Conv1D(128, 64),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GRU(256, return_sequences=True),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss=weighted_loss, 
                metrics=['accuracy'])
  return model

In [ ]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'k_fold_base_ckpt.hdf5'
checkpoint_dir_path = os.path.join(path, 'checkpoint')
checkpoint_path = os.path.join(path, 'checkpoint', ckpt_name)

# check checkpoint paht
# 경로가 없으면 생성함
if not(os.path.exists(checkpoint_dir_path)):
  os.mkdir(checkpoint_dir_path)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    )
]

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장

for i, (train, val) in enumerate(split.split(series_train, labels)):
  print('Fold', i)
  print('#'*20)

  train_dataset = make_train(series_train[train], labels[train])
  val_dataset = make_val(series_train[val], labels[val])

  model = base()

  model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=100)

  # 모델 복원
  model.load_weights(checkpoint_path)
  result = model.evaluate(val_dataset)[0]

  results.append(result)
  models.append(model)

  del model, train_dataset, val_dataset

Fold 0
####################
Epoch 1/100
22/22 [==============================] - 36s 428ms/step - loss: 0.2775 - accuracy: 0.4769 - val_loss: 0.3679 - val_accuracy: 0.4445
Epoch 2/100
22/22 [==============================] - 9s 405ms/step - loss: 0.2106 - accuracy: 0.5398 - val_loss: 0.3534 - val_accuracy: 0.5274
Epoch 3/100
22/22 [==============================] - 9s 402ms/step - loss: 0.1832 - accuracy: 0.5846 - val_loss: 0.2971 - val_accuracy: 0.5096
Epoch 4/100
22/22 [==============================] - 9s 405ms/step - loss: 0.1629 - accuracy: 0.6110 - val_loss: 0.2809 - val_accuracy: 0.5686
Epoch 5/100
22/22 [==============================] - 9s 404ms/step - loss: 0.1424 - accuracy: 0.6561 - val_loss: 0.1832 - val_accuracy: 0.5989
Epoch 6/100
22/22 [==============================] - 9s 405ms/step - loss: 0.1296 - accuracy: 0.6671 - val_loss: 0.1919 - val_accuracy: 0.5996
Epoch 7/100
22/22 [==============================] - 9s 408ms/step - loss: 0.1181 - accuracy: 0.6963 - val_loss: 

KeyboardInterrupt: ignored

In [ ]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(test_series)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [ ]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join(path, 'k-fold_base.csv')     # 저장 경로
sub.to_csv(save_path)